# 03 嵌入和向量库缓存

![image.png](imgs/RAG.png)

在上节中我们成功将文本拆分为合适大小的块，本节我们要将文本块用于嵌入模型，并生成嵌入向量。本节对应上图的5、6、7。

**嵌入(Embedding)的含义**

![图像来自网络](imgs/Word2vec.png)

langchain官方的描述是这样：
> Embedding Models take a piece of text and create a numerical representation of it.
> 
>嵌入模型获取一段文本，并创建它的数字表示。

嵌入可以理解为**向量化**，例如将文本、图像、音频等数据转换为向量。这样做的本质是**将样本映射到了高维的向量空间**，这个向量空间具有很多适应机器学习算法的性质。比如文本向量化之后可以进行距离度量(代表语义相近程度)。

![图像来自网络](imgs/Word_Embedding.png)


首先，我们先定义好一个Embedding模型。

In [29]:
import os
from langchain_community.embeddings import XinferenceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

# 首先定义一个embedding模型(这里是基于xinference的)
server_url=os.environ.get("SERVER_URL")
model_uid=os.environ.get("EMBEDDING_MODEL_UID")
embeddings_model = XinferenceEmbeddings(
    server_url=server_url,
    model_uid=model_uid
)

`langchain`的`Embedding`基类中提供了两种方法，一种用于嵌入文档，另一种用于嵌入查询。前者为`.embed_documents`，它接收多个文本作为输入，且返回多个浮点数列表；而后者`.embed_query`接收单个文本，且返回一个浮点数列表。

In [30]:
# 嵌入文档
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1024)

In [31]:
# 嵌入查询
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5]

[0.006304293405264616,
 0.011977086775004864,
 -0.025325916707515717,
 -0.02862994372844696,
 0.0062356023117899895]

为了避免重复计算，embedd可以做缓存

In [34]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryStore, LocalFileStore
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

fs = LocalFileStore("./cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model, fs, namespace=embeddings_model.model_uid
)

#缓存在嵌入之前为空
list(fs.yield_keys())

['5231650d-e76a-58ee-bae4-0efd08f4f3a6',
 'd27bc52d-cc17-5467-ac2e-258b752a4ba9']

In [35]:
# 加载一个文档并分割它（还记的上一节的内容吗）
raw_documents = TextLoader("data/index.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)  # 文本版本
documents = text_splitter.split_documents(raw_documents)  # 文档版本

# 创建矢量存储
from time import time
start = time()
db = FAISS.from_documents(documents, cached_embedder)
print(f'第一次用时：{time() - start}')

# 再次创建时会快得多，因为在缓存中无需计算embedd。
start = time()
db = FAISS.from_documents(documents, cached_embedder)
print(f'第二次用时：{time() - start}')

第一次用时：0.09300374984741211
第二次用时：0.0009961128234863281


In [36]:
#缓存中出现内容
list(fs.yield_keys())[:5]

['5231650d-e76a-58ee-bae4-0efd08f4f3a6',
 'bge-large-zh-v1.55231650d-e76a-58ee-bae4-0efd08f4f3a6',
 'bge-large-zh-v1.5d27bc52d-cc17-5467-ac2e-258b752a4ba9',
 'd27bc52d-cc17-5467-ac2e-258b752a4ba9']

In [39]:
#在内存中
store = InMemoryStore()
underlying_embeddings = embeddings_model
embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=embeddings_model.model_uid
)

start = time()
embeddings = embedder.embed_documents(["hello", "goodbye"])
print(f"第一次用时：{time()-start}")

start = time()
embeddings = embedder.embed_documents(["hello", "goodbye"])
print(f"第二次用时：{time()-start}")

第一次用时：0.06074833869934082
第二次用时：0.0
